In [1]:
# 셀 1: 필수 라이브러리 불러오기
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
import pandas as pd
from sklearn.metrics import f1_score
import numpy as np
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights


c:\Users\LENOVO\anaconda3\envs\yolo\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 지정된 프로시저를 찾을 수 없습니다'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
class MultiLabelTrafficLightDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transform

        self.label_cols = ['label_red', 'label_yellow', 'label_green', 'label_left']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = str(self.data.at[idx, 'filename']).strip()
        img_path = os.path.join(self.image_dir, img_name)

        if not os.path.exists(img_path):
            raise FileNotFoundError(f"[{idx}] 이미지 파일 없음: {img_path}")

        image = Image.open(img_path).convert("RGB")
        labels = torch.tensor(self.data.loc[idx, self.label_cols].values.astype(float), dtype=torch.float32)

        if self.transform:
            image = self.transform(image)
        return image, labels



In [3]:
# 셀 3: 전처리 정의
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])



In [4]:
# 셀 4: 절대경로 지정
base_dir = r"C:\Users\LENOVO\Documents\yolo\mobilenetdata"  # 여기를 사용자의 실제 절대경로로 변경하세요

train_csv_path = os.path.join(base_dir, "train.csv")
val_csv_path = os.path.join(base_dir, "val.csv")
train_img_dir = os.path.join(base_dir, "train_images")
val_img_dir = os.path.join(base_dir, "val_images")



In [5]:
# 셀 5: 데이터로더 정의
train_dataset = MultiLabelTrafficLightDataset(train_csv_path, train_img_dir, transform=train_transform)
val_dataset = MultiLabelTrafficLightDataset(val_csv_path, val_img_dir, transform=val_transform)
    

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)


In [6]:
# 셀 6: 모델 불러오기 및 수정

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

weights = MobileNet_V2_Weights.DEFAULT
model = mobilenet_v2(weights=weights)
model.classifier[1] = nn.Linear(model.last_channel, 4)  # 클래스 수 4개로 변경
model = model.to(device)  # GPU 또는 CPU로 모델 이동

In [7]:
train_transform = weights.transforms()
val_transform = weights.transforms()

In [8]:
# 셀 7: 손실함수 및 옵티마이저 정의
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)



In [9]:
# 셀 8: 학습 루프
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)

    avg_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")



Epoch 1/10, Loss: 0.6891
Epoch 2/10, Loss: 0.6809
Epoch 3/10, Loss: 0.6695
Epoch 4/10, Loss: 0.6478
Epoch 5/10, Loss: 0.6341
Epoch 6/10, Loss: 0.6320
Epoch 7/10, Loss: 0.6124
Epoch 8/10, Loss: 0.5993
Epoch 9/10, Loss: 0.5876
Epoch 10/10, Loss: 0.5637


In [10]:
# 셀 9: 검증 및 F1-score 측정
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        probs = torch.sigmoid(outputs).cpu()
        preds = (probs > 0.5).int()

        all_preds.append(preds)
        all_labels.append(labels)

all_preds = torch.cat(all_preds).numpy()
all_labels = torch.cat(all_labels).numpy()

f1 = f1_score(all_labels, all_preds, average='macro')
print(f"Validation F1 Score: {f1:.4f}")



Validation F1 Score: 0.1667


In [ ]:
# 셀 10: 모델 저장
torch.save(model.state_dict(), os.path.join(base_dir, "mobilenetv2_trafficlight_multilabel.pth"))
